Implementation of Byte pair encoding from scratch [Andrej Karpathy video]

A Good tokenization webapp : https://tiktokenizer.vercel.app

In [1]:
#unicode code point

ord("h")

104

In [3]:
ord("i")

105

In [4]:
[ord(x) for x in "Hello, how are you doing?"]

[72,
 101,
 108,
 108,
 111,
 44,
 32,
 104,
 111,
 119,
 32,
 97,
 114,
 101,
 32,
 121,
 111,
 117,
 32,
 100,
 111,
 105,
 110,
 103,
 63]

In [7]:
list("Hello, how are you doing?".encode("utf-8"))

[72,
 101,
 108,
 108,
 111,
 44,
 32,
 104,
 111,
 119,
 32,
 97,
 114,
 101,
 32,
 121,
 111,
 117,
 32,
 100,
 111,
 105,
 110,
 103,
 63]